In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import re  # needed for the regex pattern matching
import base64  # needed for images

from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# MongoDB connection variables
username = "aacuser"
password = "snhu123"
host = "nv-desktop-services.apporto.com"
port = 30464
db = "AAC"
collection = "animals"

# Instantiate the AnimalShelter class with all credentials
shelter = AnimalShelter(username, password, host, port, db, collection)

# Read data from MongoDB and create a DataFrame
df = pd.DataFrame.from_records(shelter.getRecordCriteria({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Debug print to check data
print(df.head())

#########################
# Dashboard Layout / View
#########################
# Initializes the Dash app
app = JupyterDash('SimpleExample')

# Encode the logo image
image_filename = '/home/armandogomez_snhu/Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Defines the layout of the app
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '200px'}), href='http://www.snhu.edu'),
    html.Center(html.B(html.H2('AG SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Button(id='button-one', n_clicks=0, children='Water Rescue'),
            html.Button(id='button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
            html.Button(id='button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
            html.Button(id='button-four', n_clicks=0, children='Reset')
        ]
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'height': '400px', 'overflowY': 'auto', 'overflowX': 'auto'},
        style_header={'backgroundColor': 'rgb(240,230,230)', 'fontWeight': 'bold'},
        style_data={'whiteSpace': 'normal', 'height': 'auto'},
        tooltip={i: {'value': i, 'use_with': 'both'} for i in df.columns},
        tooltip_delay=0,
        tooltip_duration=None,
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=True,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[0],
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6', style={'width': '40%', 'height': '200px', 'border': '2px solid black', 'display': 'inline-block'}),
                 html.Div(id='coords-box', style={'display': 'inline-block', 'verticalAlign': 'top', 'paddingLeft': '20px'}),
                 html.Div(id='button-output', style={'display': 'inline-block', 'verticalAlign': 'top', 'paddingLeft': '20px'})
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('button-output', 'children'),
     Output('graph-id', "children")],
    [Input('button-one', 'n_clicks'), Input('button-two', 'n_clicks'), 
     Input('button-three', 'n_clicks'), Input('button-four', 'n_clicks')]
)
def update_dashboard(button1, button2, button3, button4):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = 'No clicks yet'
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    button_output = "Displaying all data"
    if button_id == 'button-one':
        data = pd.DataFrame.from_records(shelter.getRecordCriteria({'$and': [
            {'$or': [{'breed': 'Labrador Retriever Mix'}, {'breed': 'Chesapeake Bay Retriever'}, {'breed': 'Newfoundland'}]},
            {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
        ]}))
        button_output = "Filtering data for Water Rescue"
    elif button_id == 'button-two':
        data = pd.DataFrame.from_records(shelter.getRecordCriteria({'$and': [
            {'$or': [{'breed': 'German Shepherd'}, {'breed': 'Alaskan Malamute'}, {'breed': 'Old English Sheepdog'}, {'breed': 'Siberian Husky'}, {'breed': 'Rottweiler'}]},
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
        ]}))
        button_output = "Filtering data for Mountain or Wilderness Rescue"
    elif button_id == 'button-three':
        data = pd.DataFrame.from_records(shelter.getRecordCriteria({'$and': [
            {'$or': [{'breed': 'Doberman Pinscher'}, {'breed': 'German Shepherd'}, {'breed': 'Golden Retriever'}, {'breed': 'Bloodhound'}, {'breed': 'Rottweiler'}]},
            {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
        ]}))
        button_output = "Filtering data for Disaster Rescue or Individual Tracking"
    else:
        data = pd.DataFrame.from_records(shelter.getRecordCriteria({}))
        button_output = "Displaying all data"
    
    if '_id' in data.columns:
        data.drop(columns=['_id'], inplace=True)
    
    # Update the pie chart
    chart = [
        dcc.Graph(
            figure=px.pie(data, names='breed', title='Percentage of breeds available')
        )
    ]
    
    return data.to_dict('records'), button_output, chart

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    [Output('map-id', "children"),
     Output('coords-box', "children")],
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(rows, selected_rows):
    markerArray = [30.75, -97.48]
    toolTip = "Austin Animal Center"
    popUpHeading = "Austin Animal Center"
    popUpParagraph = "Shelter Home Location"
    coordinates_display = "Latitude: 30.75, Longitude: -97.48"

    if selected_rows and len(selected_rows) > 0:
        selected_data = rows[selected_rows[0]]
        coordLat = selected_data['location_lat']
        coordLong = selected_data['location_long']
        
        if pd.notna(coordLat) and pd.notna(coordLong):
            coordLat = float(coordLat)
            coordLong = float(coordLong)
            markerArray = [coordLat, coordLong]
            toolTip = selected_data['breed']
            popUpHeading = selected_data['name']
            popUpParagraph = selected_data['animal_type']
            coordinates_display = f"Latitude: {coordLat}, Longitude: {coordLong}"
    
    return [dl.Map(style={'width': '100%', 'height': '200px', 'border': '2px solid black'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])], coordinates_display

# Display the Dash app
app



   rec_num age_upon_outcome animal_id animal_type                   breed  \
0        1          3 years   A746874         Cat  Domestic Shorthair Mix   
1        9          3 years   A720214         Dog  Labrador Retriever Mix   
2       10         3 months   A664290         Cat  Domestic Shorthair Mix   
3       11           1 year   A721199         Dog  Dachshund Wirehair Mix   
4        5          2 years   A691584         Dog  Labrador Retriever Mix   

         color date_of_birth             datetime            monthyear  \
0  Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1    Red/White    2013-02-04  2016-02-11 12:41:00  2016-02-11T12:41:00   
2       Tortie    2013-09-01  2013-12-08 14:58:00  2013-12-08T14:58:00   
3    Tan/White    2015-02-23  2016-02-27 17:49:00  2016-02-27T17:49:00   
4    Tan/White    2012-11-06  2015-05-30 13:48:00  2015-05-30T13:48:00   

       name outcome_subtype     outcome_type sex_upon_outcome  location_lat  \
0            